In [0]:
from google.colab import drive
drive.mount('/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /drive


In [0]:
!cp /drive/'My Drive'/retraindata/imagenet_images.rar /content/imagenet_images.rar

In [0]:
!unrar x imagenet_images.rar


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from imagenet_images.rar

Creating    imagenet_images                                           OK
Creating    imagenet_images/airbus                                    OK
Extracting  imagenet_images/airbus/n02686121_1.jpg                         0%  OK 
Extracting  imagenet_images/airbus/n02686121_10.jpg                        0%  OK 
Extracting  imagenet_images/airbus/n02686121_10086.jpg                     0%  OK 
Extracting  imagenet_images/airbus/n02686121_10087.jpg                     0%  OK 
Extracting  imagenet_images/airbus/n02686121_1011.jpg                      0%  OK 
Extracting  imagenet_images/airbus/n02686121_10117.jpg                     0%  OK 
Extracting  imagenet_images/airbus/n02686121_1012.jpg                      0%  OK 
Extracting  imagenet_images/airbus/n02686121_10185.jpg                     0%  OK 
Extracting  

In [0]:
!nvidia-smi

Thu Nov  7 10:36:47 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    31W / 250W |   7707MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import glob
import os
import random
import errno, os, stat, shutil
from tensorflow.keras.preprocessing import image


def handleRemoveReadonly(func, path, exc):
    excvalue = exc[1]
    if func in (os.rmdir, os.remove) and excvalue.errno == errno.EACCES:
        os.chmod(path, stat.S_IRWXU| stat.S_IRWXG| stat.S_IRWXO) # 0777
        func(path)
    else:
        raise
        

        
def splitit(targetpath,sourcepath,topiclist,valsplit=0.2,trainsplit=0.2):

    
    if os.path.exists(targetpath):
        shutil.rmtree(targetpath, ignore_errors=False, onerror=handleRemoveReadonly)
    os.mkdir(targetpath)
    os.mkdir(targetpath+'train_data/')
    os.mkdir(targetpath+'validation_data/')
    os.mkdir(targetpath+'test_data/')
    for topic in topiclist:
        os.mkdir(targetpath+'train_data/'+topic+'/')
        os.mkdir(targetpath+'validation_data/'+topic+'/')
        os.mkdir(targetpath+'test_data/'+topic+'/')
        fnames=glob.glob(sourcepath+topic+'/*')
        random.shuffle(fnames)
        validx=int(len(fnames)*valsplit)
        testidx=int(len(fnames)*trainsplit)+validx
        valfiles=fnames[:validx]
        testfiles=fnames[validx:testidx]
        trainfiles=fnames[testidx:]
        for file in valfiles:
            newpath=targetpath+'validation_data/'+topic+'/'+file.split('/')[-1]
            shutil.copyfile(file, newpath)
        
        for file in testfiles:
            newpath=targetpath+'test_data/'+topic+'/'+file.split('/')[-1]
            shutil.copyfile(file, newpath)
       
        for file in trainfiles:
            newpath=targetpath+'train_data/'+topic+'/'+file.split('/')[-1]
            shutil.copyfile(file, newpath)

def delifshit(sourcepath,topiclist):
    for topic in topiclist:
        fnames=glob.glob(sourcepath+topic+'/*')
        for file in fnames:
            try:
                img=image.load_img(file)
            except:
                os.remove(file)
                print(file, ' removed')

In [0]:
topics=['airbus','altar','ambulance','artillery heavy weapon gun ordnance',
        'atom bomb atomic bomb A-bomb fission bomb plutonium bomb',
        'automatic firearm automatic gun automatic weapon',
        'biplane','broad arrow','helicopter chopper whirlybird eggbeater',
        'hot-air balloon','jeep landrover','khukuri','launcher rocket launcher',
        'Model T','motor scooter scooter','revolver six-gun six-shooter',
        'Roman building','shotgun shell','sports car sport car','stealth fighter',
        'stock car','temple','totem pole','viaduct','windmill']

splitit('./sources/','./imagenet_images/',topics,0.1,0.1)

In [0]:
import tensorflow as tf

inc=tf.keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
87916544/87910968 [==============================] - 1s 0us/step


In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

x = inc.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(topics),activation='softmax')(x)

In [0]:
model = Model(inputs=inc.input, outputs=predictions)

In [0]:
for layer in inc.layers:
    layer.trainable= False

In [0]:

model.compile(optimizer='adam',metrics=['accuracy'],loss='categorical_crossentropy')

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_height=img_width=299
batch_size=64

train_datagen = ImageDataGenerator(rescale=1./255, 
                  shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen =  ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('./sources/train_data/', 
                      target_size=(img_height, img_width), 
                      batch_size=batch_size)
validation_generator = test_datagen.flow_from_directory('./sources/validation_data/', 
                      target_size=(img_height, img_width), 
                      batch_size=batch_size)


Found 14962 images belonging to 25 classes.
Found 1856 images belonging to 25 classes.


In [0]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [0]:
with tf.device('/GPU:0'):
  model.fit_generator(train_generator,steps_per_epoch=train_generator.samples//batch_size,
                      validation_data=validation_generator,
                      validation_steps=validation_generator.samples//batch_size,epochs=2)

Epoch 1/2
199/233 [========================>.....] - ETA: 38s - loss: 0.6943 - acc: 0.8003

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


231/233 [============================>.] - ETA: 2s - loss: 0.6673 - acc: 0.8072

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


233/233 [==============================] - 273s 1s/step - loss: 0.6654 - acc: 0.8074 - val_loss: 0.3849 - val_acc: 0.8831
Epoch 2/2
233/233 [==============================] - 271s 1s/step - loss: 0.4151 - acc: 0.8702 - val_loss: 0.3975 - val_acc: 0.8777


In [0]:
for layer in model.layers[:50]:
    layer.trainable= False
for layer in model.layers[50:]:
    layer.trainable= True

In [0]:
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(learning_rate=1e-2, momentum=0.9), metrics=['accuracy'], loss='categorical_crossentropy')

In [0]:
with tf.device('/GPU:0'):
  model.fit_generator(train_generator,steps_per_epoch=train_generator.samples//batch_size,
                      validation_data=validation_generator,
                      validation_steps=validation_generator.samples//batch_size,epochs=1)

 30/233 [==>...........................] - ETA: 3:57 - loss: 0.3563 - acc: 0.8906

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


148/233 [==================>...........] - ETA: 1:37 - loss: 0.4173 - acc: 0.8667

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


233/233 [==============================] - 281s 1s/step - loss: 0.4003 - acc: 0.8740 - val_loss: 0.6598 - val_acc: 0.8044


In [0]:
inc.save_weights('dataset_artificial_retrained.hdf5')